In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-8b-instant"
)




from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.codegen.com/careers")
page_data=loader.load().pop().page_content
print(page_data)

from langchain_core.prompts import PromptTemplate

# Define the prompt template
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON (NO PREAMBLE):
    """
)

# Use the prompt with a language model (llm) to extract the relevant information
chain_extract = prompt_extract | llm

# Invoke the model
res = chain_extract.invoke(input={'page_data': page_data})

# Print the response content
print(res.content)


from langchain_core.output_parsers import JsonOutputParser

# Initialize the JSON Output Parser
json_parser = JsonOutputParser()

# Parse the content of the response (the output from your model)
json_data = json_parser.parse(res.content)
print(json_data)


import pandas as pd
df=pd.read_csv("portfolio_links.csv")
df

import uuid
import chromadb

# Create a Persistent Client for ChromaDB (for persistent storage of vector data)
client = chromadb.PersistentClient('vectorstore')

# Get or create a collection in the vector store
collection = client.get_or_create_collection(name="portfolio")

# Check if the collection is empty (if no documents exist)
if not collection.count():
    # Iterate through rows in a DataFrame (df)
    for _, row in df.iterrows():
        # Add documents to the collection
        collection.add(
            documents=row["Techstack"],  # Adding the techstack from the DataFrame row
            metadatas={"links": row["Links"]},  # Adding metadata (links) from the DataFrame row
            ids=[str(uuid.uuid4())]  # Generating a unique ID for each document
        )
        


results = collection.query(
    query_texts=["Experience in Python", "Expertise in React Native"],
    n_results=2
).get('metadatas')

print(results)




Careers at Codegen | Build the Future of AI

EnterpriseSecurityPricingCareersLoginCome build the future of software engineeringFull-TimeBased in San FranciscoIn-PersonApplyOpen RolesAt Codegen, we’re on a mission to revolutionize the way enterprise companies manage and transform their codebases. We’re looking for passionate, driven individuals who are ready to take on big challenges, build cutting-edge solutions, and help shape the future of software engineering.Staff EngineerJoin Codegen as a Staff Engineer to build robust backend systems and lead projects in a dynamic, fast-paced environment. Must have 10+ years of experience, with a background in developing complex, stateful backends using Python.Applied Algorithms ResearcherJoin us as an Applied Algorithms Researcher to shape the future of AI-driven code generation. Bring your expertise in AI, particularly from a pre-LLM era, and drive the development of advanced research machines and algorithms.Compute Platform EngineerJoin Codege

In [27]:
# Query the entire collection (without any search query) to get all documents and metadata
import uuid
import chromadb

results = collection.query(
    query_texts=[""],  # Empty query will fetch all documents
    n_results=collection.count()  # Get all documents by passing the collection count
)
print(results)

# Retrieve the full list of documents and metadata
all_documents = results['documents']  # List of all documents
all_metadatas = results['metadatas']  # List of all metadata

# Print all documents and metadata
print("All Documents:", all_documents)
print("All Metadata:", all)


{'ids': [['43f62a70-2610-4fe0-b26f-aa17fd5bb0ef', '851cd383-032d-4691-8c27-9bd5aa10b4c0', '4dc64db3-2ec3-45af-bfa5-80fbc6364f11', 'b4647cbd-4a5a-44d0-8007-31e80100418f', '0a90d0f7-cf8a-47f3-b252-adfcd7f447ea', '3bdfb0d7-632a-4704-a146-ee3380c812b5', '84126f12-dab9-4d6d-b7d2-cfe4f036dccc', '2292deb8-4508-4cd1-816d-ad32abaf6fa0', '22df010d-2c06-40eb-8e10-5a25cf6203fb', '535318f7-da1d-4f73-a012-bee4e65d6ab4', 'fc378fe9-25c3-4f06-9821-b03745588936', '79e0b4df-7f36-4ad1-bdc5-fb2b1356a926', '3b768ad6-5316-42ad-9759-66caf7a722da', 'b76bc239-f374-415f-904c-a3cadafe1f1d', 'c606aeb4-4d27-470d-ae7b-1974cc27dbfb', 'eb3f7079-a007-4dc5-b431-023b2cb64267', 'bb20d8dd-7638-4c7f-bab6-908574783fb7', '66603345-7e3e-41c0-9670-c116a37c8180', '2d7ec387-4923-4fd3-88d5-7331826f4829', '8c3571d9-2ba2-4ca0-9304-9b7be5832089']], 'embeddings': None, 'documents': [['Android TV, Kotlin, Android NDK', 'Backend, Kotlin, Spring Boot', 'Java, Spring Boot, Oracle', 'DevOps, Jenkins, Docker', 'iOS, Swift, ARKit', 'Cross-pl

In [47]:
jobs=json_data

for job in jobs:
    job_skills = job['skills']  # Get the skills for the current job
    print(f"Skills for {job['role']}: {job_skills}")
    
    # Query the collection with the skills from the job
    links = collection.query(query_texts=[job_skills], n_results=2).get('metadatas', [])
    print("Links:", links)


Skills for Staff Engineer: Python, developing complex, stateful backends
Links: [[{'links': 'https://example.com/python-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]]
Skills for Applied Algorithms Researcher: AI, pre-LLM era, AI-driven code generation, advanced research machines and algorithms
Links: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}]]
Skills for Compute Platform Engineer: infrastructure, Agents, Customers
Links: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/devops-portfolio'}]]


In [51]:
from langchain.prompts import PromptTemplate

prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Navatharshini, a second-year student at the University of Moratuwa, studying Computer Science and Engineering (CSE). You are actively searching for an internship in the field of software development. 
    Your job is to write a cold email to the hiring manager regarding the job mentioned above, highlighting your academic achievements, skills, and why you are the right fit for the internship role. 
    Also, mention any relevant projects or skills from your academic work or personal portfolio that align with the company's needs.
    
    Additionally, include any relevant links to showcase your skills or projects:
    {link_list}
    
    Remember, you are Navatharshini, a motivated and eager-to-learn student seeking a valuable internship experience. Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    """
)


chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(jobs), "link_list": links})
print(res.content)

Subject: Expressed Interest in Internship Opportunities at Codegen

Dear Hiring Manager,

I am writing to express my interest in the internship opportunities available at Codegen. As a second-year student at the University of Moratuwa, studying Computer Science and Engineering (CSE), I am eager to apply my skills and knowledge in a dynamic and fast-paced environment.

With a strong foundation in programming languages, including Python, I am confident in my ability to contribute to the development of robust backend systems. My academic achievements include a strong GPA and a keen interest in software development, which has led me to participate in various projects and competitions. Some of my notable projects include:

- A machine learning portfolio (https://example.com/ml-python-portfolio) that showcases my skills in AI-driven code generation using Python.
- A DevOps portfolio (https://example.com/devops-portfolio) that highlights my expertise in infrastructure management and deploymen

In [41]:
print(job)

[{'role': 'Staff Engineer', 'experience': '10+ years', 'skills': 'Python, developing complex, stateful backends', 'description': 'Join Codegen as a Staff Engineer to build robust backend systems and lead projects in a dynamic, fast-paced environment.'}, {'role': 'Applied Algorithms Researcher', 'experience': '', 'skills': 'AI, pre-LLM era, AI-driven code generation, advanced research machines and algorithms', 'description': 'Join us as an Applied Algorithms Researcher to shape the future of AI-driven code generation. Bring your expertise in AI, particularly from a pre-LLM era, and drive the development of advanced research machines and algorithms.'}, {'role': 'Compute Platform Engineer', 'experience': '', 'skills': 'infrastructure, Agents, Customers', 'description': 'Join Codegen as a Senior/Staff Compute Platform Engineer to build and maintain infrastructure used by our Agents and Customers.'}]


In [3]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

# Set up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run headlessly without opening a browser window
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the page
driver.get("https://wso2.com/careers/")

# Wait for the JavaScript to load (you might need to adjust the time)
time.sleep(5)  # Wait for the content to load, adjust the sleep time if necessary

# Extract the page content
page_content = driver.page_source

# Print or process the content
print(page_content)

# Close the driver
driver.quit()


In [6]:
pip install webdriver-manager
